# NOAA stations

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [3]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Read data from NOAA

In [6]:
# https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn

In [7]:
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

In [12]:
stations = pd.read_fwf(
    "data/raw/ghcnd-stations.txt",
    header=0,
    names=["station_id", "lat", "lon", "unk", "name", "end", "end1", "end2"],
)

In [19]:
lax = stations[stations["station_id"] == "USW00023174"]

In [20]:
inventory = pd.read_fwf(
    "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt",
    names=["station_id", "lat", "lon", "product", "begin", "end"],
)

In [21]:
# inventory["product"].value_counts()

In [25]:
lax_inventory = inventory[inventory["station_id"] == "USW00023174"]

In [26]:
lax_inventory

,station_id,lat,lon,product,begin,end
689722,USW00023174,33.9381,118.3889,TMAX,1944,2021
689723,USW00023174,33.9381,118.3889,TMIN,1944,2021
689724,USW00023174,33.9381,118.3889,PRCP,1944,2021
689725,USW00023174,33.9381,118.3889,SNOW,1944,2019
689726,USW00023174,33.9381,118.3889,SNWD,1944,2019
689727,USW00023174,33.9381,118.3889,ACMH,1965,1997
689728,USW00023174,33.9381,118.3889,ACSH,1965,1997
689729,USW00023174,33.9381,118.3889,AWND,1984,2021
689730,USW00023174,33.9381,118.3889,FMTM,1995,2012
689731,USW00023174,33.9381,118.3889,FRGT,1962,1962
